In [ ]:
import pandas as pd
import numpy as np
import os
import copy
import math
#from google.colab import drive
#drive.mount("/content/gdrive", force_remount = True)

In [ ]:
data_path = os.path.join(
                   os.getcwd(),
                   'dataset'
)

# To Label the Data with 3rd class (Pre-FOG)

In [ ]:
def label_prefog(dataset,window_length = 4):
  dataset.drop(index = list(dataset[dataset['Action'] == 0].index),inplace=True)
  window_length = 64*window_length
    
  fog_index=[]
  for i in dataset.index: 
      if dataset.loc[i,'Action'] == 2:
        fog_index.append(i)
  fog_index



  start_indices=[]
  for i in fog_index:
    if (dataset.loc[i-1,'Action']!=dataset.loc[i,'Action']):
      start_indices.append(i)
 

  prefog=[]
  for start in start_indices:
    prefog_start = [x for x in range(start-window_length,start)]
    prefog.append(prefog_start)

  prefog = [item for sublist in prefog for item in sublist]

  for i in prefog:
       dataset.loc[i,'Action'] = 3
  dataset['Action'] = dataset['Action'] - 1
  return dataset

In [ ]:
people = []
for person in os.listdir(data_path):
    if '.txt' in person: 
        people.append(person)
for window_length in range(1,5):
    dataset = pd.DataFrame()
    for person in people: 
        name = person.split('R')[0]
        print (name)
        file = data_path+"\\"+person
        temp = pd.read_csv(file,delimiter= " ", header = None)
        print (person,' is read',end = '\t')
        if 2 in temp[max(temp.columns)].unique():
            print ('Adding {} to dataset'.format(person),end = '\t')
            temp.columns = ['time','A_F','A_V','A_L','L_F','L_V','L_L','T_F','T_V','T_L','Action']    
            temp = label_prefog(temp,window_length).reset_index(drop=True)
            temp['name'] = name
            print ('{} is labelled'.format(person))
            dataset = pd.concat([dataset,temp],axis = 0)

        print ('')
    dataset.reset_index(drop =True,inplace=True) 
    to_path = data_path + "\\raw_labelled"
    to_name = to_path +"\\win_"+str(window_length)+".csv"
     
    dataset.to_csv(to_name,index = False)


display(dataset.head())

# To Extract Non-Overlapping windows of length w *f_s from the continously logged accelerometer data from the dataset.

In [ ]:
def create_window(act,window_length,dataframe):
    
  indices = list(dataframe[dataframe.Action == act].index)
  groups = []
  temp = []
  group_count = 0
  for i in range(len(indices)):
    if i == len(indices)-1:
      temp.append(indices[i])
      groups.append(temp)
      temp = []
      break
    temp.append(indices[i])
    if indices[i]+1 != indices[i+1]: 
      group_count+=1
      groups.append(temp)
      temp = []

  fs = 64
  window_length = 4
  # window_length = window_length*fs

  final_dataframe = pd.DataFrame()
  for i in groups: 
    required = math.floor(len(i)/(window_length*fs))
    
    req_index = i[0:(required*fs)]
    
    final_dataframe = pd.concat([final_dataframe,dataframe.iloc[req_index,:]],axis = 0)
  return final_dataframe


In [ ]:
for window_length in range(1,5):
  
  path = os.getcwd()+"/dataset"
  name = path+"/raw_labelled/win_"+str(window_length)+".csv"
  dataframe = pd.read_csv(name)

  activities = []
  for act in range(3):
    activities.append(create_window(act,window_length,dataframe))
  to_write = pd.concat(activities,axis = 0)
  to_path = path + "/windows"+"/windowed_"+str(window_length)+".csv"
  to_write.to_csv(to_path,index = False)

# Extracting Features

In [ ]:
#read file 
window_length = 4
fs = 64
# for window_length in range(1,5):
w = window_length*fs
FE_path = path + "/windows/windowed_"
name = FE_path + str(window_length) + ".csv"
dataframe = pd.read_csv(name)

df = dataframe.drop(columns=['time','Action','name'])
stat = pd.DataFrame()


col= list(df.columns)
for s in col:    
  print (s)
  mn =[] 
  var = []
  std = []
  mav = []
  rms =[]
  for i in range(0,len(df),w):
      mn_  = np.mean(df[s].iloc[i:i+w])
      var_  = np.var(df[s].iloc[i:i+w])
      std_  = np.std(df[s].iloc[i:i+w])
      mav_  = np.mean(abs(df[s].iloc[i:i+w]))
      rms_  = np.sqrt(np.mean((df[s].iloc[i:i+w])**2))

      mn.append(mn_)
      var.append(var_)
      std.append(std_)
      mav.append(mav_)
      rms.append(rms_)

  stat['mean_'+s] = mn
  stat['var_'+s] = var
  stat['std_'+s] = std
  stat['rms_'+s] = rms
  stat['mav_'+s] = mav


stat.shape

import copy
stat1 = copy.copy(stat)
stat1['w'] = dataframe['Action'].iloc[[x for x in range(0,len(dataframe),w)]].to_list()
order = ['w']
order += stat1.columns.to_list()[:-1]
stat1 = stat1[order]
stat1.columns
col = stat1.columns.to_list()
col[0] = 0
stat1.columns = col
feature_name = path + "/features/time_"+str(window_length)+".csv"
stat1.to_csv(feature_name, index = False)

In [ ]:
df.shape